In [1]:
import laspy
import smrf
import numpy as np
import pandas as pd
import rasterio

Read in some data sourced from a YellowScan, courtesy of the StREAM Lab at Virginia Tech.

This record is approximately 54 million points.

In [2]:
fn = '../../smrf_data/StREAM_20170405.laz'
las = laspy.read(fn)

Apply SRMF to remove the vegetation.  Although the point density of scan is quite high (approximately 430 points per square meter), it isn't necessary that SMRF's cellsize corresponds to this value.  A 1 meter grid will often work just as well, and run significantly faster.  A DTM can be generated from the classified output in the next step at whatever resolution is appropriate, and need not be the same as is used here.

Because this area has very few large buildings and is dominated by relatively small vegetation, a very small window can be used.  Even large trees don't require a radius setting here that's very large, because the lidar will typically penetrate through the canopy, and the minimum return for that grid cell will often still be ground.

AGH is above ground height, which can be used to classify vegetation.  We'll use some easy breakpoints to define low, medium, and high vegetation for this, but there are likely much more sophisticated ways to do this.

In [3]:
windows = 2 # in pixels
slope_threshold = .15
elevation_threshold = .1
elevation_scaler = 0
cellsize = 1


Zsmrf,Tsmrf,obj_cells,obj_array,AGH = smrf.smrf(*np.array((las.x,las.y,las.z)),cellsize,windows,
                                    slope_threshold,elevation_threshold,
                                    elevation_scaler,return_AGH=True)

In [4]:
new_file = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
new_file.points = las.points
new_file.classification[obj_array==0] = 2
new_file.classification[obj_array==1] = 4  # Assume everything else is medium vegetation
new_file.classification[AGH<1] = 3         # Unless under a meter, then it's low vegetation
new_file.classification[AGH>3] = 5         # Or higher than 3 meters, then it's high vegetation
new_file.write(fn[:-4] + '.classified.laz')

In [9]:
np.array(las.z) - AGH

array([0.08478146, 0.01578153, 0.11212136, ..., 0.05902864, 0.07928742,
       0.04823845])

In [8]:
AGH

array([604.58221854, 604.47521847, 604.58587864, ..., 607.47197136,
       607.34571258, 607.43476155])